In [4]:
# install Hugging Face Libraries
!pip install  git+https://github.com/huggingface/peft.git
!pip install "transformers==4.26.1" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr
!pip install --quiet sentencepiece
!git clone https://github.com/nunziati/bert-vs-t5-for-question-answering/

%cd bert-vs-t5-for-question-answering
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-gzawx1wu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-gzawx1wu
  Resolved https://github.com/huggingface/peft.git to commit 632997d1fb776c3cf05d8c2537ac9a98a7ce9435
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'bert-vs-t5-for-question-answering'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 144 (delta 73), reused 93 (delta 38), pack-reused 0
Receiving objects: 100% (144/144), 74.45 KiB | 8.27 MiB/s, done.
Resolving deltas: 100% (73/73), done.
/

In [5]:
import torch
from transformers import AutoModelForSeq2SeqLM,T5Tokenizer
from peft import get_peft_config, get_peft_model, TaskType,PrefixTuningConfig,PeftModelForSeq2SeqLM,PeftModel
from __future__ import print_function
from typing import List, Tuple
from tqdm import tqdm
import torch

from datasets import load_dataset
from transformers import PreTrainedTokenizer, T5ForConditionalGeneration, T5Tokenizer, AdamW, set_seed
from torch.utils.data import DataLoader
import argparse
from MyDataset import Dataset
import MyDataset


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_sa

In [6]:
# Downloading model and tokenizer

model_name_or_path = 't5-small'
# model_name_or_path = 'google/mt5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
tokenizer = T5Tokenizer.from_pretrained(model_name_or_path)

In [7]:
# Defining config for prefix tuning with ParameterEfficientFineTuning Library

peft_config=PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    num_virtual_tokens=20)

model = get_peft_model(model, peft_config)

In [8]:
# Function to train model with required hyperparameters
def train(model, tokenizer, optimizer, train_set, validation_set, num_train_epochs, device, batch_size, max_input_length: int = 512):

    my_trainset_dataloader = DataLoader(train_set, batch_size=batch_size,
                                        num_workers=10, collate_fn=lambda data: train_set.pack_minibatch(data))
    my_validation_dataloader = DataLoader(validation_set, batch_size=batch_size,
                                          num_workers=10, collate_fn=lambda data: validation_set.pack_minibatch(data))

    # set training mode on the model
    model.train()

    # model to device
    model.to(device)

    f1_old: int = 0
    j=0
    print("ck1")
    for epoch in range(num_train_epochs):
        epoch_train_loss = 0.
        for contexts,questions,answers in tqdm(my_trainset_dataloader):
            optimizer.zero_grad()

            inputs = list(map(lambda tuple: f"question:{tuple[0]}  context:{tuple[1]}", zip(questions,contexts)))
            encoded_inputs = tokenizer(
                                    inputs,
                                    padding="longest",
                                    max_length=max_input_length,
                                    truncation=True,
                                    return_tensors="pt",
                                )
            encoded_targets = tokenizer(
                                    answers,
                                    padding="longest",
                                    max_length=max_input_length,
                                    truncation=True,
                                    return_tensors="pt",
                                )

            input_ids, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
            encoded_targets = encoded_targets.input_ids

            # replace padding target token id's of the labels by -100, crossEntropy skip target label == -100
            encoded_targets[encoded_targets == tokenizer.pad_token_id] = -100

            input_ids = input_ids.to(device)
            encoded_targets = encoded_targets.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=encoded_targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item() * batch_size
        print(f"epoch={epoch + 1}/{num_train_epochs}")
        print(f"\t Train loss = {epoch_train_loss/len(train_set):.4f}")

        model.eval()
        with torch.no_grad():
            model_predictions_encoded = []
            target_encoded = []
            for contexts, questions, answers in tqdm(my_validation_dataloader):
                inputs = list(map(lambda tuple: f"question: {tuple[0]}  context:{tuple[1]}", zip(
                    questions, contexts)))
                encoded_inputs = tokenizer(
                    inputs,
                    padding="longest",
                    max_length=max_input_length,
                    truncation=True,
                    return_tensors="pt",
                )
                encoded_targets = tokenizer(
                    answers,
                    padding="longest",
                    max_length=max_input_length,
                    truncation=True,
                    return_tensors="pt",
                )
                encoded_inputs, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
                encoded_targets = encoded_targets.input_ids

                encoded_inputs = encoded_inputs.to(device)
                encoded_targets = encoded_targets.to(device)
                attention_mask = attention_mask.to(device)
                model_predictions = model.generate(
                    input_ids=encoded_inputs, attention_mask=attention_mask)

                model_predictions_encoded += model_predictions.tolist()
                target_encoded += encoded_targets.tolist()
        f1, exact_match = validation_set.evaluate(model_predictions_encoded, target_encoded)

        print(f"\t Validation F1 = {f1:.2f}, EM = {exact_match:.2f}")
        if f1 > f1_old :
            model.save_pretrained(f'results/{model.name_or_path}/model/best-f1')
            tokenizer.save_pretrained(f'results/{model.name_or_path}/tokenizer/best-f1')
            f1_old = f1
        if epoch+1 % 10 == 0:
            model.save_pretrained(f'results/{model.name_or_path}/model/checkpoint-{epoch+1}')
            tokenizer.save_pretrained(f'results/{model.name_or_path}/tokenizer/checkpoint-{epoch+1}')
        model.train()

    model.save_pretrained(
        f'results/{model.name_or_path}/model/checkpoint-{epoch+1}')
    tokenizer.save_pretrained(
        f'results/{model.name_or_path}/tokenizer/checkpoint-{epoch+1}')

In [9]:
# Set seed
set_seed(7)

# creating the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [10]:
# load dataset from the hub
ds1 = load_dataset('tydiqa', 'secondary_task')

# print(ds1['train'])

Computing checksums:  50%|#####     | 1/2 [00:06<00:06,  6.98s/it]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/49881 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5077 [00:00<?, ? examples/s]

Dataset tydiqa downloaded and prepared to /root/.cache/huggingface/datasets/tydiqa/secondary_task/1.0.0/b8a6c4c0db10bf5703d7b36645e5dbae821b8c0e902dac9daeecd459a8337148. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# extracting the english language examples from the tydiqa dataset
english_examples = [example for example in ds1['train'] if example['id'][0:7] == 'english']                # train set
print(len(english_examples))

english_examples_val = [example for example in ds1['validation'] if example['id'][0:7] == 'english']       # validation set
print(len(english_examples_val))

3696
440


In [12]:
from datasets import Dataset as ds

# Convert the TydiQA dataset to a Hugging Face Dataset object
def convert_to_dataset(data):
    dataset = {"context": [], "question": [], "answers": []}
    for article in data:
        question = article["question"]
        context = article["context"]
        dataset["context"].append(context)
        dataset["question"].append(question)
        dataset["answers"].append(article['answers'])
    return ds.from_dict(dataset)


In [13]:
dataset_train = convert_to_dataset(english_examples)
dataset_valid = convert_to_dataset(english_examples_val)

# Tokenizing the training and validation examples
train_set_en= Dataset(dataset_train,tokenizer, parser=MyDataset.DatasetMap.squad)
val_set_en= Dataset(dataset_valid,tokenizer, parser=MyDataset.DatasetMap.squad)



  0%|          | 0/3696 [00:00<?, ?it/s]

100%|██████████| 3696/3696 [00:00<00:00, 19405.29it/s]


100%|██████████| 440/440 [00:00<00:00, 19052.81it/s]


In [14]:
# Trainable parameters for prompt tuning, keeping the model parameters freezed
model.print_trainable_parameters()

trainable params: 122880 || all params: 60629504 || trainable%: 0.20267360260773368


In [15]:
# training and validation

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_en,
      validation_set=val_set_en,
       device=device,batch_size= batch_size,num_train_epochs=7)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


ck1




  0%|          | 0/924 [00:00<?, ?it/s]

  0%|          | 1/924 [00:04<1:06:17,  4.31s/it]

  0%|          | 3/924 [00:04<17:52,  1.16s/it]  

  1%|          | 5/924 [00:04<09:03,  1.69it/s]

  1%|          | 8/924 [00:04<04:36,  3.32it/s]

  1%|          | 11/924 [00:04<02:54,  5.24it/s]

  2%|▏         | 14/924 [00:04<02:01,  7.50it/s]

  2%|▏         | 17/924 [00:05<01:29, 10.16it/s]

  2%|▏         | 20/924 [00:05<01:09, 12.93it/s]

  2%|▏         | 23/924 [00:05<00:58, 15.39it/s]

  3%|▎         | 26/924 [00:05<00:50, 17.71it/s]

  3%|▎         | 29/924 [00:05<00:44, 19.99it/s]

  3%|▎         | 32/924 [00:05<00:40, 21.83it/s]

  4%|▍         | 35/924 [00:05<00:39, 22.33it/s]

  4%|▍         | 38/924 [00:05<00:38, 23.17it/s]

  4%|▍         | 41/924 [00:05<00:37, 23.78it/s]

  5%|▍         | 44/924 [00:06<00:38, 22.69it/s]

  5%|▌         | 47/924 [00:06<00:41, 20.99it/s]

  5%|▌         | 50/924 [00:06<00:42, 20.67it/s]

  6%|▌         | 53/924 [00:06<00:43, 20.10it/s]

  6%|▌ 

epoch=1/7
	 Train loss = 0.7545




  0%|          | 0/172 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  1%|          | 1/172 [00:00<01:39,  1.72it/s]

  2%|▏         | 3/172 [00:00<00:35,  4.80it/s]

  3%|▎         | 5/172 [00:00<00:26,  6.29it/s]

  3%|▎         | 6/172 [00:01<00:27,  6.00it/s]

  4%|▍         | 7/172 [00:01<00:27,  5.97it/s]

  5%|▌         | 9/172 [00:01<00:20,  8.14it/s]

  6%|▌         | 10/172 [00:01<00:22,  7.07it/s]

  7%|▋         | 12/172 [00:01<00:19,  8.24it/s]

  8%|▊         | 13/172 [00:01<00:19,  8.01it/s]

  8%|▊         | 14/172 [00:02<00:19,  8.09it/s]

  9%|▉         | 16

...

	 Validation F1 = 66.28, EM = 46.21




  0%|          | 0/924 [00:00<?, ?it/s]

  0%|          | 1/924 [00:00<07:38,  2.01it/s]

  0%|          | 4/924 [00:00<01:55,  7.94it/s]

  1%|          | 7/924 [00:00<01:11, 12.78it/s]

  1%|          | 10/924 [00:00<00:58, 15.57it/s]

  1%|▏         | 13/924 [00:00<00:50, 17.88it/s]

  2%|▏         | 16/924 [00:01<00:44, 20.21it/s]

  2%|▏         | 19/924 [00:01<00:41, 21.90it/s]

  2%|▏         | 22/924 [00:01<00:39, 22.99it/s]

  3%|▎         | 25/924 [00:01<00:38, 23.58it/s]

  3%|▎         | 28/924 [00:01<00:36, 24.64it/s]

  3%|▎         | 31/924 [00:01<00:35, 25.22it/s]

  4%|▎         | 34/924 [00:01<00:36, 24.29it/s]

  4%|▍         | 37/924 [00:01<00:35, 25.02it/s]

  4%|▍         | 40/924 [00:02<00:34, 25.69it/s]

  5%|▍         | 43/924 [00:02<00:36, 24.20it/s]

  5%|▍         | 46/924 [00:02<00:35, 24.48it/s]

  5%|▌         | 49/924 [00:02<00:34, 25.19it/s]

  6%|▌         | 52/924 [00:02<00:34, 25.26it/s]

  6%|▌         | 55/924 [00:02<00:35, 24.74it/s]

  6%|▋    

epoch=7/7
	 Train loss = 0.7271




  0%|          | 0/172 [00:00<?, ?it/s]

  1%|          | 1/172 [00:00<01:42,  1.67it/s]

  2%|▏         | 3/172 [00:00<00:36,  4.63it/s]

  3%|▎         | 5/172 [00:00<00:27,  6.15it/s]

  3%|▎         | 6/172 [00:01<00:28,  5.83it/s]

  4%|▍         | 7/172 [00:01<00:28,  5.77it/s]

  5%|▌         | 9/172 [00:01<00:21,  7.45it/s]

  6%|▌         | 10/172 [00:01<00:24,  6.65it/s]

  7%|▋         | 12/172 [00:01<00:19,  8.18it/s]

  8%|▊         | 13/172 [00:02<00:20,  7.95it/s]

  8%|▊         | 14/172 [00:02<00:19,  8.01it/s]

  9%|▉         | 16/172 [00:02<00:16,  9.38it/s]

 10%|▉         | 17/172 [00:02<00:17,  9.00it/s]

 11%|█         | 19/172 [00:02<00:16,  9.55it/s]

 12%|█▏        | 20/172 [00:02<00:18,  8.38it/s]

 12%|█▏        | 21/172 [00:02<00:20,  7.38it/s]

 13%|█▎        | 23/172 [00:03<00:20,  7.33it/s]

 14%|█▍        | 24/172 [00:03<00:22,  6.61it/s]

 15%|█▌        | 26/172 [00:03<00:17,  8.48it/s]

 16%|█▌        | 27/172 [00:03<00:17,  8.41it/s]

 17%|█▋      

	 Validation F1 = 66.59, EM = 46.50


In [16]:
# Zero-Shot Training(No training) and Validation
def train_zeroshot(model, tokenizer, optimizer, train_set, validation_set, num_train_epochs, device, batch_size, max_input_length: int = 512):

    my_trainset_dataloader = DataLoader(train_set, batch_size=batch_size,
                                        num_workers=10, collate_fn=lambda data: train_set.pack_minibatch(data))
    my_validation_dataloader = DataLoader(validation_set, batch_size=batch_size,
                                          num_workers=10, collate_fn=lambda data: validation_set.pack_minibatch(data))

    # set training mode on the model
    model.train()

    # model to device
    model.to(device)

    f1_old: int = 0
    j=0
    print("ck1")
    for epoch in range(num_train_epochs):
        epoch_train_loss = 0.
        for contexts,questions,answers in tqdm(my_trainset_dataloader):
            optimizer.zero_grad()
            j=j+1
            if(j>1):
              break
            inputs = list(map(lambda tuple: f"question:{tuple[0]}  context:{tuple[1]}", zip(questions,contexts)))
            encoded_inputs = tokenizer(
                                    inputs,
                                    padding="longest",
                                    max_length=max_input_length,
                                    truncation=True,
                                    return_tensors="pt",
                                )
            encoded_targets = tokenizer(
                                    answers,
                                    padding="longest",
                                    max_length=max_input_length,
                                    truncation=True,
                                    return_tensors="pt",
                                )

            input_ids, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
            encoded_targets = encoded_targets.input_ids

            # replace padding target token id's of the labels by -100, crossEntropy skip target label == -100
            encoded_targets[encoded_targets == tokenizer.pad_token_id] = -100

            input_ids = input_ids.to(device)
            encoded_targets = encoded_targets.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=encoded_targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item() * batch_size
        print(f"epoch={epoch + 1}/{num_train_epochs}")
        print(f"\t Train loss = {epoch_train_loss/len(train_set):.4f}")

        model.eval()
        with torch.no_grad():
            model_predictions_encoded = []
            target_encoded = []
            for contexts, questions, answers in tqdm(my_validation_dataloader):
                inputs = list(map(lambda tuple: f"question: {tuple[0]}  context:{tuple[1]}", zip(
                    questions, contexts)))
                encoded_inputs = tokenizer(
                    inputs,
                    padding="longest",
                    max_length=max_input_length,
                    truncation=True,
                    return_tensors="pt",
                )
                encoded_targets = tokenizer(
                    answers,
                    padding="longest",
                    max_length=max_input_length,
                    truncation=True,
                    return_tensors="pt",
                )
                encoded_inputs, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
                encoded_targets = encoded_targets.input_ids

                encoded_inputs = encoded_inputs.to(device)
                encoded_targets = encoded_targets.to(device)
                attention_mask = attention_mask.to(device)
                model_predictions = model.generate(
                    input_ids=encoded_inputs, attention_mask=attention_mask)

                model_predictions_encoded += model_predictions.tolist()
                target_encoded += encoded_targets.tolist()
        f1, exact_match = validation_set.evaluate(model_predictions_encoded, target_encoded)

        print(f"\t Validation F1 = {f1:.2f}, EM = {exact_match:.2f}")
        if f1 > f1_old :
            model.save_pretrained(f'results/{model.name_or_path}/model/best-f1')
            tokenizer.save_pretrained(f'results/{model.name_or_path}/tokenizer/best-f1')
            f1_old = f1
        if epoch+1 % 10 == 0:
            model.save_pretrained(f'results/{model.name_or_path}/model/checkpoint-{epoch+1}')
            tokenizer.save_pretrained(f'results/{model.name_or_path}/tokenizer/checkpoint-{epoch+1}')
        model.train()

    model.save_pretrained(
        f'results/{model.name_or_path}/model/checkpoint-{epoch+1}')
    tokenizer.save_pretrained(
        f'results/{model.name_or_path}/tokenizer/checkpoint-{epoch+1}')

In [17]:
# Few-Shot Learning(train with few examples) and Validation
def train_fewshot(model, tokenizer, optimizer, train_set, validation_set, num_train_epochs, device, batch_size, max_input_length: int = 512):

    my_trainset_dataloader = DataLoader(train_set, batch_size=batch_size,
                                        num_workers=10, collate_fn=lambda data: train_set.pack_minibatch(data))
    my_validation_dataloader = DataLoader(validation_set, batch_size=batch_size,
                                          num_workers=10, collate_fn=lambda data: validation_set.pack_minibatch(data))

    # set training mode on the model
    model.train()

    # model to device
    model.to(device)

    f1_old: int = 0
    print("ck1")
    for epoch in range(num_train_epochs):
        epoch_train_loss = 0.
        j=0
        for contexts,questions,answers in tqdm(my_trainset_dataloader):
            optimizer.zero_grad()
            j=j+1
            if(j>150):
              break
            inputs = list(map(lambda tuple: f"question:{tuple[0]}  context:{tuple[1]}", zip(questions,contexts)))
            encoded_inputs = tokenizer(
                                    inputs,
                                    padding="longest",
                                    max_length=max_input_length,
                                    truncation=True,
                                    return_tensors="pt",
                                )
            encoded_targets = tokenizer(
                                    answers,
                                    padding="longest",
                                    max_length=max_input_length,
                                    truncation=True,
                                    return_tensors="pt",
                                )

            input_ids, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
            encoded_targets = encoded_targets.input_ids

            # replace padding target token id's of the labels by -100, crossEntropy skip target label == -100
            encoded_targets[encoded_targets == tokenizer.pad_token_id] = -100

            input_ids = input_ids.to(device)
            encoded_targets = encoded_targets.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=encoded_targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item() * batch_size
        print(f"epoch={epoch + 1}/{num_train_epochs}")
        print(f"\t Train loss = {epoch_train_loss/len(train_set):.4f}")

        model.eval()
        with torch.no_grad():
            model_predictions_encoded = []
            target_encoded = []
            for contexts, questions, answers in tqdm(my_validation_dataloader):
                inputs = list(map(lambda tuple: f"question: {tuple[0]}  context:{tuple[1]}", zip(
                    questions, contexts)))
                encoded_inputs = tokenizer(
                    inputs,
                    padding="longest",
                    max_length=max_input_length,
                    truncation=True,
                    return_tensors="pt",
                )
                encoded_targets = tokenizer(
                    answers,
                    padding="longest",
                    max_length=max_input_length,
                    truncation=True,
                    return_tensors="pt",
                )
                encoded_inputs, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
                encoded_targets = encoded_targets.input_ids

                encoded_inputs = encoded_inputs.to(device)
                encoded_targets = encoded_targets.to(device)
                attention_mask = attention_mask.to(device)
                model_predictions = model.generate(
                    input_ids=encoded_inputs, attention_mask=attention_mask)

                model_predictions_encoded += model_predictions.tolist()
                target_encoded += encoded_targets.tolist()
        f1, exact_match = validation_set.evaluate(model_predictions_encoded, target_encoded)

        print(f"\t Validation F1 = {f1:.2f}, EM = {exact_match:.2f}")
        if f1 > f1_old :
            model.save_pretrained(f'results/{model.name_or_path}/model/best-f1')
            tokenizer.save_pretrained(f'results/{model.name_or_path}/tokenizer/best-f1')
            f1_old = f1
        if epoch+1 % 10 == 0:
            model.save_pretrained(f'results/{model.name_or_path}/model/checkpoint-{epoch+1}')
            tokenizer.save_pretrained(f'results/{model.name_or_path}/tokenizer/checkpoint-{epoch+1}')
        model.train()

    model.save_pretrained(
        f'results/{model.name_or_path}/model/checkpoint-{epoch+1}')
    tokenizer.save_pretrained(
        f'results/{model.name_or_path}/tokenizer/checkpoint-{epoch+1}')

**Russian**

In [18]:
# Preprocessing the russian examples from the TyDiQA dataset
russian_examples = [example for example in ds1['train'] if example['id'][0:7] == 'russian']
print(len(russian_examples))

russian_examples_val = [example for example in ds1['validation'] if example['id'][0:7] == 'russian']
print(len(russian_examples_val))

dataset_train_rn = convert_to_dataset(russian_examples)
dataset_valid_rn = convert_to_dataset(russian_examples_val)

train_set_rn= Dataset(dataset_train_rn,tokenizer, parser=MyDataset.DatasetMap.squad)
val_set_rn= Dataset(dataset_valid_rn,tokenizer, parser=MyDataset.DatasetMap.squad)

6490
812




  0%|          | 0/6490 [00:00<?, ?it/s]

 16%|█▋        | 1064/6490 [00:00<00:00, 10632.86it/s]

 33%|███▎      | 2128/6490 [00:00<00:00, 10125.41it/s]

 49%|████▊     | 3162/6490 [00:00<00:00, 10217.34it/s]

 64%|██████▍   | 4185/6490 [00:00<00:00, 10126.80it/s]

 80%|████████  | 5199/6490 [00:00<00:00, 9959.71it/s] 

100%|██████████| 6490/6490 [00:00<00:00, 9884.28it/s]


100%|██████████| 812/812 [00:00<00:00, 10206.98it/s]


**Zero Shot Learning: Russian**

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_zeroshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_rn,
      validation_set=val_set_rn,
       device=device,batch_size= batch_size,num_train_epochs=1)

ck1




  0%|          | 0/1623 [00:00<?, ?it/s]

  0%|          | 1/1623 [00:01<29:23,  1.09s/it]


epoch=1/1
	 Train loss = 0.0006




  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 1/361 [00:00<04:13,  1.42it/s]

  1%|          | 2/361 [00:00<02:40,  2.24it/s]

  1%|          | 3/361 [00:01<02:04,  2.87it/s]

  1%|          | 4/361 [00:01<01:33,  3.82it/s]

  1%|▏         | 5/361 [00:01<01:25,  4.15it/s]

  2%|▏         | 6/361 [00:01<01:22,  4.30it/s]

  2%|▏         | 7/361 [00:01<01:20,  4.41it/s]

  2%|▏         | 8/361 [00:02<01:16,  4.59it/s]

  2%|▏         | 9/361 [00:02<01:15,  4.64it/s]

  3%|▎         | 10/361 [00:02<01:16,  4.58it/s]

  3%|▎         | 11/361 [00:02<01:15,  4.61it/s]

  3%|▎         | 12/361 [00:03<01:14,  4.65it/s]

  4%|▎         | 13/361 [00:03<01:09,  5.04it/s]

  4%|▍         | 14/361 [00:03<01:06,  5.21it/s]

  4%|▍         | 16/361 [00:03<00:58,  5.91it/s]

  5%|▍         | 17/361 [00:03<00:58,  5.91it/s]

  5%|▌         | 19/361 [00:04<00:55,  6.18it/s]

  6%|▌         | 20/361 [00:04<00:53,  6.37it/s]

  6%|▌         | 21/361 [00:04<00:50,  6.70it/s]

  6%|▌         |

	 Validation F1 = 33.76, EM = 7.62


**Few Shot Learning: Russian**

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_fewshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_rn,
      validation_set=val_set_rn,
       device=device,batch_size= batch_size,num_train_epochs=7)

ck1




  0%|          | 0/1623 [00:00<?, ?it/s]

  0%|          | 1/1623 [00:00<13:34,  1.99it/s]

  0%|          | 3/1623 [00:00<05:03,  5.34it/s]

  0%|          | 5/1623 [00:00<03:25,  7.89it/s]

  0%|          | 8/1623 [00:00<02:21, 11.38it/s]

  1%|          | 10/1623 [00:01<02:10, 12.40it/s]

  1%|          | 12/1623 [00:01<02:02, 13.13it/s]

  1%|          | 14/1623 [00:01<01:54, 14.05it/s]

  1%|          | 16/1623 [00:01<01:51, 14.36it/s]

  1%|          | 18/1623 [00:01<01:44, 15.30it/s]

  1%|          | 20/1623 [00:01<01:44, 15.29it/s]

  1%|▏         | 22/1623 [00:01<01:44, 15.29it/s]

  1%|▏         | 24/1623 [00:01<01:48, 14.68it/s]

  2%|▏         | 26/1623 [00:02<01:50, 14.50it/s]

  2%|▏         | 28/1623 [00:02<01:48, 14.70it/s]

  2%|▏         | 30/1623 [00:02<01:50, 14.42it/s]

  2%|▏         | 32/1623 [00:02<01:46, 14.92it/s]

  2%|▏         | 34/1623 [00:02<01:44, 15.26it/s]

  2%|▏         | 36/1623 [00:02<01:42, 15.52it/s]

  2%|▏         | 38/1623 [00:02<01:39, 15.

epoch=1/7
	 Train loss = 0.1164




  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 1/361 [00:00<05:56,  1.01it/s]

  1%|          | 2/361 [00:01<03:35,  1.67it/s]

  1%|          | 3/361 [00:01<02:50,  2.10it/s]

  1%|          | 4/361 [00:01<02:09,  2.76it/s]

  1%|▏         | 5/361 [00:02<01:59,  2.98it/s]

  2%|▏         | 6/361 [00:02<01:45,  3.38it/s]

  2%|▏         | 7/361 [00:02<01:35,  3.70it/s]

  2%|▏         | 8/361 [00:02<01:28,  3.99it/s]

  2%|▏         | 9/361 [00:02<01:23,  4.19it/s]

  3%|▎         | 10/361 [00:03<01:21,  4.29it/s]

  3%|▎         | 11/361 [00:03<01:19,  4.40it/s]

  3%|▎         | 12/361 [00:03<01:16,  4.56it/s]

  4%|▎         | 13/361 [00:03<01:11,  4.88it/s]

  4%|▍         | 14/361 [00:03<01:08,  5.09it/s]

  4%|▍         | 16/361 [00:04<00:59,  5.82it/s]

  5%|▍         | 17/361 [00:04<00:59,  5.81it/s]

  5%|▌         | 19/361 [00:04<00:54,  6.27it/s]

  6%|▌         | 20/361 [00:04<00:53,  6.39it/s]

  6%|▌         | 21/361 [00:04<00:50,  6.69it/s]

  6%|▌         |

	 Validation F1 = 34.06, EM = 8.10


...

	 Validation F1 = 35.31, EM = 9.28




  0%|          | 0/1623 [00:00<?, ?it/s]

  0%|          | 1/1623 [00:00<20:18,  1.33it/s]

  0%|          | 3/1623 [00:00<06:59,  3.86it/s]

  0%|          | 5/1623 [00:01<04:24,  6.11it/s]

  0%|          | 7/1623 [00:01<03:11,  8.44it/s]

  1%|          | 9/1623 [00:01<02:35, 10.36it/s]

  1%|          | 11/1623 [00:01<02:20, 11.46it/s]

  1%|          | 13/1623 [00:01<02:10, 12.32it/s]

  1%|          | 15/1623 [00:01<02:01, 13.19it/s]

  1%|          | 17/1623 [00:01<01:51, 14.41it/s]

  1%|          | 19/1623 [00:01<01:49, 14.64it/s]

  1%|▏         | 21/1623 [00:02<01:50, 14.56it/s]

  1%|▏         | 23/1623 [00:02<01:46, 14.96it/s]

  2%|▏         | 25/1623 [00:02<01:53, 14.04it/s]

  2%|▏         | 27/1623 [00:02<01:53, 14.01it/s]

  2%|▏         | 29/1623 [00:02<01:50, 14.41it/s]

  2%|▏         | 31/1623 [00:02<01:52, 14.12it/s]

  2%|▏         | 33/1623 [00:02<01:46, 14.90it/s]

  2%|▏         | 35/1623 [00:03<01:47, 14.77it/s]

  2%|▏         | 37/1623 [00:03<01:42, 15.4

epoch=7/7
	 Train loss = 0.1122




  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 1/361 [00:00<05:13,  1.15it/s]

  1%|          | 2/361 [00:01<03:10,  1.88it/s]

  1%|          | 3/361 [00:01<02:28,  2.42it/s]

  1%|          | 4/361 [00:01<01:51,  3.20it/s]

  1%|▏         | 5/361 [00:01<01:49,  3.25it/s]

  2%|▏         | 6/361 [00:02<01:47,  3.30it/s]

  2%|▏         | 7/361 [00:02<01:49,  3.23it/s]

  2%|▏         | 8/361 [00:02<01:49,  3.23it/s]

  2%|▏         | 9/361 [00:03<01:51,  3.17it/s]

  3%|▎         | 10/361 [00:03<01:52,  3.13it/s]

  3%|▎         | 11/361 [00:03<01:51,  3.13it/s]

  3%|▎         | 12/361 [00:04<01:40,  3.48it/s]

  4%|▎         | 13/361 [00:04<01:31,  3.79it/s]

  4%|▍         | 14/361 [00:04<01:26,  4.01it/s]

  4%|▍         | 15/361 [00:04<01:11,  4.83it/s]

  4%|▍         | 16/361 [00:04<01:09,  4.95it/s]

  5%|▍         | 17/361 [00:04<01:06,  5.16it/s]

  5%|▌         | 19/361 [00:05<00:59,  5.73it/s]

  6%|▌         | 20/361 [00:05<00:56,  6.06it/s]

  6%|▌         |

	 Validation F1 = 35.31, EM = 9.35


**Indonesian**

In [21]:
# Preprocessing Indonesian examples from TyDiQA dataset
indonesian_examples = [example for example in ds1['train'] if example['id'][0:10] == 'indonesian']
print(len(indonesian_examples))

indonesian_examples_val = [example for example in ds1['validation'] if example['id'][0:10] == 'indonesian']
print(len(indonesian_examples_val))

dataset_train_in = convert_to_dataset(indonesian_examples)
dataset_valid_in = convert_to_dataset(indonesian_examples_val)

train_set_in= Dataset(dataset_train_in,tokenizer, parser=MyDataset.DatasetMap.squad)
val_set_in= Dataset(dataset_valid_in,tokenizer, parser=MyDataset.DatasetMap.squad)

5702
565




  0%|          | 0/5702 [00:00<?, ?it/s]

 34%|███▍      | 1938/5702 [00:00<00:00, 19378.45it/s]

100%|██████████| 5702/5702 [00:00<00:00, 18058.95it/s]


100%|██████████| 565/565 [00:00<00:00, 17267.81it/s]


**Zero Shot Learning: Indonesian**

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_zeroshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_rn,
      validation_set=val_set_rn,
       device=device,batch_size= batch_size,num_train_epochs=1)

ck1




  0%|          | 0/1623 [00:00<?, ?it/s]

  0%|          | 1/1623 [00:00<22:03,  1.23it/s]


epoch=1/1
	 Train loss = 0.0004




  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 1/361 [00:00<03:49,  1.57it/s]

  1%|          | 2/361 [00:00<02:26,  2.45it/s]

  1%|          | 3/361 [00:01<01:56,  3.08it/s]

  1%|          | 4/361 [00:01<01:27,  4.06it/s]

  1%|▏         | 5/361 [00:01<01:20,  4.40it/s]

  2%|▏         | 6/361 [00:01<01:19,  4.48it/s]

  2%|▏         | 7/361 [00:01<01:18,  4.54it/s]

  2%|▏         | 8/361 [00:02<01:15,  4.68it/s]

  2%|▏         | 9/361 [00:02<01:15,  4.65it/s]

  3%|▎         | 10/361 [00:02<01:16,  4.60it/s]

  3%|▎         | 11/361 [00:02<01:16,  4.59it/s]

  3%|▎         | 12/361 [00:02<01:15,  4.64it/s]

  4%|▎         | 13/361 [00:03<01:14,  4.69it/s]

  4%|▍         | 14/361 [00:03<01:14,  4.67it/s]

  4%|▍         | 16/361 [00:03<01:02,  5.52it/s]

  5%|▍         | 17/361 [00:03<01:01,  5.58it/s]

  5%|▌         | 19/361 [00:04<00:56,  6.02it/s]

  6%|▌         | 20/361 [00:04<00:54,  6.31it/s]

  6%|▌         | 21/361 [00:04<00:50,  6.69it/s]

  6%|▌         |

	 Validation F1 = 35.31, EM = 9.35


**Few Shot Learning: Indonesian**

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_fewshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_rn,
      validation_set=val_set_rn,
       device=device,batch_size= batch_size,num_train_epochs=7)

ck1




  0%|          | 0/1623 [00:00<?, ?it/s]

  0%|          | 1/1623 [00:00<14:19,  1.89it/s]

  0%|          | 3/1623 [00:00<05:09,  5.23it/s]

  0%|          | 5/1623 [00:00<03:28,  7.75it/s]

  0%|          | 8/1623 [00:00<02:23, 11.28it/s]

  1%|          | 10/1623 [00:01<02:10, 12.33it/s]

  1%|          | 12/1623 [00:01<02:05, 12.87it/s]

  1%|          | 14/1623 [00:01<01:56, 13.84it/s]

  1%|          | 16/1623 [00:01<01:52, 14.29it/s]

  1%|          | 18/1623 [00:01<01:45, 15.20it/s]

  1%|          | 20/1623 [00:01<01:45, 15.20it/s]

  1%|▏         | 22/1623 [00:01<01:44, 15.35it/s]

  1%|▏         | 24/1623 [00:02<01:49, 14.64it/s]

  2%|▏         | 26/1623 [00:02<01:51, 14.38it/s]

  2%|▏         | 28/1623 [00:02<01:48, 14.68it/s]

  2%|▏         | 30/1623 [00:02<01:50, 14.36it/s]

  2%|▏         | 32/1623 [00:02<01:46, 14.98it/s]

  2%|▏         | 34/1623 [00:02<01:43, 15.34it/s]

  2%|▏         | 36/1623 [00:02<01:41, 15.63it/s]

  2%|▏         | 38/1623 [00:02<01:39, 15.

epoch=1/7
	 Train loss = 0.1095




  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 1/361 [00:00<03:37,  1.65it/s]

  1%|          | 2/361 [00:00<02:15,  2.65it/s]

  1%|          | 3/361 [00:01<01:47,  3.34it/s]

  1%|          | 4/361 [00:01<01:22,  4.30it/s]

  1%|▏         | 5/361 [00:01<01:20,  4.42it/s]

  2%|▏         | 6/361 [00:01<01:18,  4.51it/s]

  2%|▏         | 7/361 [00:01<01:17,  4.55it/s]

  2%|▏         | 8/361 [00:02<01:16,  4.64it/s]

  2%|▏         | 9/361 [00:02<01:15,  4.64it/s]

  3%|▎         | 10/361 [00:02<01:15,  4.65it/s]

  3%|▎         | 11/361 [00:02<01:15,  4.64it/s]

  3%|▎         | 12/361 [00:02<01:14,  4.71it/s]

  4%|▎         | 13/361 [00:03<01:13,  4.72it/s]

  4%|▍         | 14/361 [00:03<01:13,  4.72it/s]

  4%|▍         | 16/361 [00:03<01:01,  5.60it/s]

  5%|▍         | 17/361 [00:03<01:00,  5.66it/s]

  5%|▌         | 19/361 [00:04<00:55,  6.16it/s]

  6%|▌         | 20/361 [00:04<00:53,  6.40it/s]

  6%|▌         | 21/361 [00:04<00:50,  6.73it/s]

  6%|▌         |

	 Validation F1 = 35.59, EM = 9.76


...

	 Validation F1 = 36.57, EM = 11.22


  9%|▉         | 150/1623 [00:10<01:45, 13.99it/s]


epoch=7/7
	 Train loss = 0.1034


100%|██████████| 361/361 [01:09<00:00,  5.21it/s]
1444it [00:00, 42536.31it/s]

	 Validation F1 = 36.52, EM = 11.50


**Telugu**

In [24]:
# Preprocessing Telugu examples from TyDiQA dataset

telugu_examples = [example for example in ds1['train'] if example['id'][0:6] == 'telugu']
print(len(telugu_examples))

telugu_examples_val = [example for example in ds1['validation'] if example['id'][0:6] == 'telugu']
print(len(telugu_examples_val))

dataset_train_te = convert_to_dataset(telugu_examples)
dataset_valid_te = convert_to_dataset(telugu_examples_val)

train_set_te= Dataset(dataset_train_te,tokenizer, parser=MyDataset.DatasetMap.squad)
val_set_te= Dataset(dataset_valid_te,tokenizer, parser=MyDataset.DatasetMap.squad)

5563
669


100%|██████████| 669/669 [00:00<00:00, 17252.87it/s]


**Zero Shot Learning: Telugu**

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_zeroshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_te,
      validation_set=val_set_te,
       device=device,batch_size= batch_size,num_train_epochs=1)

ck1


  0%|          | 1/1391 [00:01<23:55,  1.03s/it]


epoch=1/1
	 Train loss = 0.0014


100%|██████████| 319/319 [01:06<00:00,  4.83it/s]
1273it [00:00, 77319.12it/s]

	 Validation F1 = 23.23, EM = 2.28


**Few Shot Learning: Telugu**

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_fewshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_te,
      validation_set=val_set_te,
       device=device,batch_size= batch_size,num_train_epochs=7)

ck1


 11%|█         | 150/1391 [00:08<01:08, 18.04it/s]


epoch=1/7
	 Train loss = 0.1688


100%|██████████| 319/319 [01:06<00:00,  4.80it/s]
1273it [00:00, 77151.53it/s]


	 Validation F1 = 23.48, EM = 2.44


...

	 Validation F1 = 25.85, EM = 4.79


 11%|█         | 150/1391 [00:07<01:04, 19.37it/s]


epoch=7/7
	 Train loss = 0.1582


100%|██████████| 319/319 [01:03<00:00,  5.03it/s]
1273it [00:00, 68903.72it/s]

	 Validation F1 = 26.01, EM = 5.11


**Arabic**

In [27]:
# Preprocessing Arabic examples from TyDiQA dataset
arabic_examples = [example for example in ds1['train'] if example['id'][0:6] == 'arabic']
print(len(telugu_examples))

arabic_examples_val = [example for example in ds1['validation'] if example['id'][0:6] == 'arabic']
print(len(arabic_examples_val))

dataset_train_ar = convert_to_dataset(arabic_examples)
dataset_valid_ar = convert_to_dataset(arabic_examples_val)

train_set_ar= Dataset(dataset_train_ar,tokenizer, parser=MyDataset.DatasetMap.squad)
val_set_ar= Dataset(dataset_valid_ar,tokenizer, parser=MyDataset.DatasetMap.squad)

5563
921


100%|██████████| 921/921 [00:00<00:00, 13095.47it/s]


**Zero Shot Learning: Arabic**

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_zeroshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_ar,
      validation_set=val_set_ar,
       device=device,batch_size= batch_size,num_train_epochs=1)

ck1


  0%|          | 1/3702 [00:00<53:13,  1.16it/s]


epoch=1/1
	 Train loss = 0.0001


100%|██████████| 405/405 [01:24<00:00,  4.80it/s]
1620it [00:00, 38824.61it/s]

	 Validation F1 = 44.23, EM = 4.94


**Few Shot Learning: Arabic**

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_fewshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_ar,
      validation_set=val_set_ar,
       device=device,batch_size= batch_size,num_train_epochs=7)

ck1


  4%|▍         | 150/3702 [00:07<02:54, 20.36it/s]


epoch=1/7
	 Train loss = 0.0272


100%|██████████| 405/405 [01:23<00:00,  4.86it/s]
1620it [00:00, 60254.97it/s]


	 Validation F1 = 43.89, EM = 5.12


...

	 Validation F1 = 44.13, EM = 6.36


  4%|▍         | 150/3702 [00:08<03:28, 17.06it/s]


epoch=7/7
	 Train loss = 0.0265


100%|██████████| 405/405 [01:22<00:00,  4.90it/s]
1620it [00:00, 35084.82it/s]

	 Validation F1 = 44.33, EM = 6.60


**Finnish**

In [30]:
# Preprocessing Finnish examples from TyDiQA dataset

finnish_examples = [example for example in ds1['train'] if example['id'][0:7] == 'finnish']
print(len(finnish_examples))

finnish_examples_val = [example for example in ds1['validation'] if example['id'][0:7] == 'finnish']
print(len(finnish_examples_val))

dataset_train_fn = convert_to_dataset(finnish_examples)
dataset_valid_fn = convert_to_dataset(finnish_examples_val)

train_set_fn= Dataset(dataset_train_fn,tokenizer, parser=MyDataset.DatasetMap.squad)
val_set_fn= Dataset(dataset_valid_fn,tokenizer, parser=MyDataset.DatasetMap.squad)

6855
782


100%|██████████| 782/782 [00:00<00:00, 17008.64it/s]


**Zero Shot Learning: Finnish**

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_zeroshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_fn,
      validation_set=val_set_fn,
       device=device,batch_size= batch_size,num_train_epochs=1)

ck1


  0%|          | 1/1714 [00:00<25:03,  1.14it/s]


epoch=1/1
	 Train loss = 0.0003


100%|██████████| 328/328 [00:59<00:00,  5.48it/s]
1309it [00:00, 57797.35it/s]

	 Validation F1 = 30.38, EM = 9.85


**Few Shot Learning: Finnish**

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_fn,
      validation_set=val_set_fn,
       device=device,batch_size= batch_size,num_train_epochs=7)

ck1


100%|██████████| 1714/1714 [01:40<00:00, 17.10it/s]


epoch=1/7
	 Train loss = 0.9107


100%|██████████| 328/328 [00:59<00:00,  5.50it/s]
1309it [00:00, 52205.46it/s]


	 Validation F1 = 30.10, EM = 9.85


...

	 Validation F1 = 31.58, EM = 11.46


100%|██████████| 1714/1714 [01:38<00:00, 17.35it/s]


epoch=7/7
	 Train loss = 0.7983


100%|██████████| 328/328 [00:59<00:00,  5.51it/s]
1309it [00:00, 42072.57it/s]

	 Validation F1 = 31.47, EM = 11.46


**Bengali**

In [33]:
# Preprocessing bengali examples from TyDiQA dataset

bengali_examples = [example for example in ds1['train'] if example['id'][0:7] == 'bengali']
print(len(bengali_examples))

bengali_examples_val = [example for example in ds1['validation'] if example['id'][0:7] == 'bengali']
print(len(bengali_examples_val))

dataset_train_bn = convert_to_dataset(bengali_examples)
dataset_valid_bn = convert_to_dataset(bengali_examples_val)

train_set_bn= Dataset(dataset_train_bn,tokenizer, parser=MyDataset.DatasetMap.squad)
val_set_bn= Dataset(dataset_valid_bn,tokenizer, parser=MyDataset.DatasetMap.squad)

2390
113


100%|██████████| 113/113 [00:00<00:00, 12809.63it/s]


**Zero Shot Learning: Bengali**

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_zeroshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_bn,
      validation_set=val_set_bn,
       device=device,batch_size= batch_size,num_train_epochs=1)

ck1


  0%|          | 1/598 [00:00<08:44,  1.14it/s]


epoch=1/1
	 Train loss = 0.0009


100%|██████████| 49/49 [00:17<00:00,  2.78it/s]
193it [00:00, 52316.98it/s]

	 Validation F1 = 39.24, EM = 4.15


**Few Shot Learning: Bengali**

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4
train_fewshot(model=model,
      tokenizer=tokenizer,
      optimizer=optimizer,
      train_set=train_set_bn,
      validation_set=val_set_bn,
       device=device,batch_size= batch_size,num_train_epochs=7)

ck1


 25%|██▌       | 150/598 [00:08<00:24, 18.59it/s]


epoch=1/7
	 Train loss = 0.1541


100%|██████████| 49/49 [00:11<00:00,  4.27it/s]
193it [00:00, 50819.30it/s]


	 Validation F1 = 41.63, EM = 8.81


...

	 Validation F1 = 59.66, EM = 28.50


 25%|██▌       | 150/598 [00:08<00:24, 18.50it/s]


epoch=7/7
	 Train loss = 0.1293


100%|██████████| 49/49 [00:07<00:00,  6.98it/s]
193it [00:00, 60016.36it/s]

	 Validation F1 = 61.78, EM = 31.09


In [37]:
import pickle
with open('my_trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)